In [8]:
from keras.preprocessing import image_dataset_from_directory
import os
import numpy as np

In [9]:
import cv2
from tensorflow.keras.preprocessing.image import img_to_array,ImageDataGenerator
from tensorflow.keras.utils import to_categorical, plot_model
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D 
from keras.layers import Dense, Dropout, Activation, Flatten ,BatchNormalization,GlobalMaxPool2D 
from keras.layers.experimental.preprocessing import Rescaling
from keras.optimizers import Adam
import glob,random
from keras import backend as K
import keras
import tensorflow as tf

In [10]:
train_dir = "F:/WorkSpace/Data/ASL/new_ASL_dataset/egde"
test_dir = "F:/WorkSpace/Data/ASL/new_ASL_dataset/test"

In [11]:
trclasses , teclasses  =  os.listdir(train_dir), os.listdir(test_dir)
print(str(trclasses)+"\n"+str(teclasses))

['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'A']
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [12]:
def preprocess(img):
    scale = 2
    delta = 0
    ddepth = cv2.CV_16S
    grad_x = cv2.Sobel(img, ddepth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
    grad_y = cv2.Sobel(img, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)
    edges = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    out=np.expand_dims(edges,2).astype('float32')
    return out

In [13]:
count=0
data=[]
labels=[]

In [21]:
aug = ImageDataGenerator(
    width_shift_range=(0.1),
    height_shift_range=(0.1),
    zoom_range=(0.1),
    rotation_range=8,
    horizontal_flip=False,
    rescale=1./255,
    #preprocessing_function=preprocess
    )
test_datagen = ImageDataGenerator(rescale=1./255)#preprocessing_function=preprocess)

In [22]:
val_data=test_datagen.flow_from_directory(
    test_dir,
    class_mode="categorical",
    classes=teclasses,
    color_mode='grayscale',
    batch_size=32,
    target_size=(240, 240),
    interpolation="bilinear",
    shuffle=True,
    
)

Found 165 images belonging to 26 classes.


In [23]:
train_data=aug.flow_from_directory(
    train_dir,
    class_mode='categorical',
    classes=teclasses,
    color_mode='grayscale',
    batch_size=32,
    target_size=(240, 240),
    interpolation="bilinear"   , 
    shuffle=True
)

Found 3811 images belonging to 26 classes.


In [24]:
for image_batch, labels_batch in train_data:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 240, 240, 1)
(32, 26)


In [25]:
model=Sequential()

model.add(Conv2D(64, (3,3), input_shape = (240, 240, 1), activation = 'relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))


model.add(Conv2D(256, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(26, activation = 'softmax'))


model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 238, 238, 64)      640       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 119, 119, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 117, 117, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 58, 58, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 26, 26, 128)      

In [26]:
opt = keras.optimizers.Adam(learning_rate=1e-2)

model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [30]:
model.fit(
  train_data,
  validation_data=val_data,
  epochs=50
)

Epoch 1/50
120/120 [==============================] - 28s 235ms/step - loss: 0.4127 - accuracy: 0.8719 - val_loss: 9.0769 - val_accuracy: 0.0606
Epoch 2/50
120/120 [==============================] - 28s 232ms/step - loss: 0.3587 - accuracy: 0.8824 - val_loss: 7.6264 - val_accuracy: 0.0364
Epoch 3/50
120/120 [==============================] - 28s 234ms/step - loss: 0.3653 - accuracy: 0.8767 - val_loss: 8.9305 - val_accuracy: 0.0424
Epoch 4/50
120/120 [==============================] - 28s 234ms/step - loss: 0.3689 - accuracy: 0.8827 - val_loss: 6.6467 - val_accuracy: 0.0667
Epoch 5/50
120/120 [==============================] - 28s 233ms/step - loss: 0.3531 - accuracy: 0.8856 - val_loss: 7.6001 - val_accuracy: 0.0606
Epoch 6/50
120/120 [==============================] - 28s 232ms/step - loss: 0.3200 - accuracy: 0.8903 - val_loss: 9.0700 - val_accuracy: 0.0364
Epoch 7/50
120/120 [==============================] - 28s 232ms/step - loss: 0.3360 - accuracy: 0.8908 - val_loss: 11.0515 - val_a

In [31]:
save_dir="F:/WorkSpace/Data/ASL/new_ASL_dataset/temp_model/tot_data_edge/II"

In [32]:
model.save(save_dir)

INFO:tensorflow:Assets written to: F:/WorkSpace/Data/ASL/new_ASL_dataset/temp_model/tot_data_edge/II\assets
